# Set up EarthScope data downloads

Gareth Funning, University of California, Riverside

If we want to do things with GNSS data in the US, many of those things rely on being able to download data from EarthScope. This notebook is intended to show how to download things using your EarthScope credentials.

## 0. What do you mean, you don't have EarthScope credentials?!

The [EarthScope Consortium](https://www.earthscope.org/) is a non-profit corporation that exists to serve the scientific community with geophysical data, instrumentation and training. Formed in 2023 by a merger between two previous long-lived community organizations, the Incorporated Research Institutions for Seismology ([IRIS](https://en.wikipedia.org/wiki/IRIS_Consortium)) and the University NAVSTAR Corporation ([UNAVCO](https://en.wikipedia.org/wiki/UNAVCO)), EarthScope is the current operator of the [SAGE](https://www.iris.edu/hq/sage) (seismology) and [GAGE](https://www.unavco.org/what-we-do/gage-facility/) (geodesy) facilities for the National Science Foundation.

For the purposes of this course, EarthScope maintains comprehensive data archives of GNSS and SAR data and data products, and we will want to use them! Which means that we will need to have an account with EarthScope (available for free!) in order to download these things.

**To register for an EarthScope account**, all you have to do is go here: https://www.earthscope.org/user/ and answer a few questions. Choose an appropriate password, and don't forget it (or use a password manager)!

There, that wasn't so difficult, was it? Let's get on with setting up scripted data downloads using your newly-established credentials...

## 1. Import dependencies

If you don't have these things installed, Jupyter will be sad.

In [ ]:
# import these things that you should have installed with the GEO244 conda environment

import os
from pathlib import Path
import requests
from earthscope_sdk.auth.device_code_flow import DeviceCodeFlowSimple
from earthscope_sdk.auth.auth_flow import NoTokensError

## 2. Choose a location for your single sign-on token

EarthScope has implemented a token-based access system for its data servers, so that once you have this established, you do not need to log in over and over again.

In [ ]:
# choose where you want the token saved - the default file name is sso_tokens.json
# if you want to keep the default name, set the path to a directory. Include a file name to rename. 

# YOU DO NEED TO GIVE AN APPROPRIATE LOCATION FOR YOUR MACHINE
token_path = "/path/to/your/token/directory"  # THIS IS A FAKE NAME!
# example: "/home/gareth/pkgs/tokens"

url = "https://gage-data.earthscope.org/path/to/the/file/you/want.yeah" # THIS IS A FAKE NAME!
# example: "https://gage-data.earthscope.org/archive/gnss/products/velocity/cwu.final_nam14.vel"

In [ ]:
# OPTIONAL: make a directory to download the token into

if not os.path.exists(token_path):
    os.makedirs(token_path)
else:
    print("Directory "+token_path+ " already exists!")

## 3. Establish a token if you need one

Hopefully this will do nothing if you already have your token set up. If you haven't set it up, it will do some things!

**If this code cell outputs a weblink beneath the code**, you should click on it and confirm that the text strings match!

In [ ]:
# instantiate the device code flow subclass
device_flow = DeviceCodeFlowSimple(Path(token_path))
try:
    # get access token from local path
    device_flow.get_access_token_refresh_if_necessary()
except NoTokensError:
    # if no token was found locally, do the device code flow
    device_flow.do_flow()
token = device_flow.access_token

In [ ]:
# check that the token file you wanted actually downloaded

os.listdir(token_path)

## 4. Download the file using your token

If everything is set up properly, we can use the token you've established on your machine to download the file you wanted!

In [ ]:
# request a file and provide the token in the Authorization header
file_name = Path(url).name
directory_to_save_file = Path.cwd() # where you want to save the downloaded file 

r = requests.get(url, headers={"authorization": f"Bearer {token}"})
if r.status_code == requests.codes.ok:
    # save the file
    with open(Path(directory_to_save_file /
 file_name), 'wb') as f:
        for data in r:
            f.write(data)
else:
    #problem occured
    print(f"failure: {r.status_code}, {r.reason}")

In [ ]:
# OPTIONAL: check to see if the download worked!

os.listdir(".")

## 5. Acknowledgements

This was hacked together, with minimum alterations, from the [example codes](https://gitlab.com/earthscope/public/earthscope-sdk) provided by the EarthScope Data Services team. Thank you all!